In [1]:
import pandas as pd
from datetime import datetime, date

pd.set_option("display.max_columns", None)

df = pd.read_csv(r'C:\Users\imffb\project\EDA\car\AllData_new_used.csv',low_memory=False)
df = df.drop(columns=['Unnamed: 0'])
df['year_month'] = pd.to_datetime(df['scraped_first_time'].str.split('-').str[:2].str.join('-'), format='%Y-%m')
display(df)

df['new=0, certified=1, used=2'] = df['new=0, certified=1, used=2'].str.replace('Certified Pre-Owned', '1').astype(float)
df_new = df[df['new=0, certified=1, used=2'] < 1]
df_used = df[df['new=0, certified=1, used=2'] >= 1]

df_x = pd.DataFrame(columns=['Descriptor','New','Used']).set_index('Descriptor')

,scraped_first_time,make,model,year,kilometers,price,province,City,Vehicle_information.transmission,Vehicle_information.drivetrain,Vehicle_information.body_style,Vehicle_information.exterior_colour,Vehicle_information.fuel_type,Vehicle_information.trim,"new=0, certified=1, used=2",year_month
0,2021-05-17,Volkswagen,Jetta,2019,74277,18840,BC,Abbotsford,Automatic,FWD,Sedan,White,Gas,Comfortline,2.0,2021-05-01
1,2020-12-16,Volkswagen,Arteon,2021,123,54845,BC,Maple Ridge,Automatic,AWD,Sedan,Silver,Gas,Execline,0.0,2020-12-01
2,2021-06-16,Toyota,Highlander,2019,39278,42888,BC,Duncan,Automatic,AWD,Suv,Black,Gas,Xle,2.0,2021-06-01
3,2021-07-08,Chevrolet,Tahoe,2021,40,74143,SK,Saskatoon,Automatic,4WD,Suv,Black,Diesel,Rst,0.0,2021-07-01
4,2021-07-09,Jeep,Compass,2018,37072,27999,MB,Winnipeg,Automatic,4WD,Suv,Grey,Gas,Trailhawk,2.0,2021-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871063,2022-02-04,Ram,1500,2022,13,75055,AB,Calgary,Automatic,4WD,Pickup,White,Gas,Sport,0,2022-02-01
871064,2021-12-14,Ford,Super Duty F 350,2022,4,71690,AB,Edmonton,Automatic,4WD,Pickup,Black,Gas,Xlt,0,2021-12-01
871065,2022-02-04,Volkswagen,Golf Sportwagen,2019,90,35075,AB,Edmonton,Manual,FWD,Wagon,Black,Gas,Highline,2,2022-02-01
871066,2021-12-30,Nissan,Rogue,2021,8,38958,AB,Medicine Hat,Automatic,AWD,Suv,White,Gas,Sv,0,2021-12-01


In [2]:
from statistics import mean
import datetime as dt

df_new_prices = df_new.groupby('year_month')['price'].mean()
year_averages1 = df_new.groupby(pd.PeriodIndex(df_new['year_month'], freq='Y'))['price'].mean()

df_used_prices = df_used.groupby('year_month')['price'].mean()
year_averages2 = df_used.groupby(pd.PeriodIndex(df_used['year_month'], freq='Y'))['price'].mean()

df_x.loc['TOTAL average','New'] = '{:.2f}'.format(df_new['price'].mean())
df_x.loc['TOTAL average','Used'] = '{:.2f}'.format(df_used['price'].mean())
df_x.loc['TOTAL %MOM','New'] = '{:.2f}'.format(mean([100.0 * a1 / a2 - 100 for a1, a2 in zip(df_new_prices.values[1:], df_new_prices.values)]))
df_x.loc['TOTAL %MOM','Used'] = '{:.2f}'.format(mean([100.0 * a1 / a2 - 100 for a1, a2 in zip(df_used_prices.values[1:], df_used_prices.values)]))
df_x.loc['TOTAL %YOY 2022-2021','New'] = '{:.2f}'.format((year_averages1.loc['2022'] / year_averages1.loc['2021'] - 1)*100, '\n')
df_x.loc['TOTAL %YOY 2022-2021','Used'] = '{:.2f}'.format((year_averages2.loc['2022'] / year_averages2.loc['2021'] - 1)*100)

today = dt.date.today()
first = today.replace(day=1)
lastMonth = first - dt.timedelta(days=1)
last_month = lastMonth.strftime("%Y-%m-01")

df_x.loc['{} average'.format(datetime.strptime(last_month, "%Y-%m-%d").strftime("%B")),'New'] = '{:.2f}'.format(df_new[df_new['year_month']==last_month]['price'].mean())
df_x.loc['{} average'.format(datetime.strptime(last_month, "%Y-%m-%d").strftime("%B")),'Used'] = '{:.2f}'.format(df_used[df_used['year_month']==last_month]['price'].mean())

display(df_x)

,New,Used
Descriptor,,
TOTAL average,55339.12,31403.62
TOTAL %MOM,2.75,7.00
TOTAL %YOY 2022-2021,13.25,23.84
May average,64099.29,35724.15


In [3]:
#Body styles
for n in ['New','Used']:
    if n == 'New':
        df_n = df_new.copy()
    elif n == 'Used':
        df_n = df_used.copy()

    for body in df_n['Vehicle_information.body_style'].unique():
        if n == 'New':
            df_n = df_new[df_new['Vehicle_information.body_style']==body].copy()
        elif n == 'Used':
            df_n = df_used[df_used['Vehicle_information.body_style']==body].copy()
        year_averages = df_n.groupby(pd.PeriodIndex(df_n['year_month'], freq='Y'))['price'].mean()
        
        if n == 'New':
            df_n = df_new[df_new['year_month']==last_month].copy()
        elif n == 'Used':
            df_n = df_used[df_used['year_month']==last_month].copy()           
        df_n = df_n[df_n['Vehicle_information.body_style']==body]
        
        df_x.loc['{} average - {}'.format(body, datetime.strptime(last_month, "%Y-%m-%d").strftime("%B")), n] = '{:.2f}'.format(df_n['price'].mean())
        df_x.loc['{} %YOY 2022-2021'.format(body), n] = '{:.2f}'.format((year_averages.loc['2022'] / year_averages.loc['2021'] - 1)*100)
        
display(df_x)

,New,Used
Descriptor,,
TOTAL average,55339.12,31403.62
TOTAL %MOM,2.75,7.00
TOTAL %YOY 2022-2021,13.25,23.84
May average,64099.29,35724.15
Sedan average - May,42797.05,25264.24
Sedan %YOY 2022-2021,14.00,36.23
Suv average - May,57062.15,37050.48
Suv %YOY 2022-2021,9.82,21.19
Van/Minivan average - May,49735.10,28179.84


In [4]:
#Body styles
for n in ['New','Used']:
    if n == 'New':
        df_n = df_new.copy()
    elif n == 'Used':
        df_n = df_used.copy()

    for province in df_n['province'].unique():
        if n == 'New':
            df_n = df_new[df_new['province']==province].copy()
        elif n == 'Used':
            df_n = df_used[df_used['province']==province].copy()
        year_averages = df_n.groupby(pd.PeriodIndex(df_n['year_month'], freq='Y'))['price'].mean()
        
        if n == 'New':
            df_n = df_new[df_new['year_month']==last_month].copy()
        elif n == 'Used':
            df_n = df_used[df_used['year_month']==last_month].copy()
        df_n = df_n[df_n['province']==province]
        
        df_x.loc['{} average - {}'.format(province, datetime.strptime(last_month, "%Y-%m-%d").strftime("%B")), n] = '{:.2f}'.format(df_n['price'].mean())
        df_x.loc['{} %YOY 2022-2021'.format(province), n] = '{:.2f}'.format((year_averages.loc['2022'] / year_averages.loc['2021'] - 1)*100)
        
display(df_x)

,New,Used
Descriptor,,
TOTAL average,55339.12,31403.62
TOTAL %MOM,2.75,7.00
TOTAL %YOY 2022-2021,13.25,23.84
May average,64099.29,35724.15
Sedan average - May,42797.05,25264.24
Sedan %YOY 2022-2021,14.00,36.23
Suv average - May,57062.15,37050.48
Suv %YOY 2022-2021,9.82,21.19
Van/Minivan average - May,49735.10,28179.84


In [5]:
df_x.to_csv(r'C:\Users\imffb\project\EDA\car\car_data.csv', index=True)